In [1]:
import numpy as np
import pandas as pd
import queue
import matplotlib.pyplot as plt

import sys
backtest_dir = 'C://backtest/backtest/'
if backtest_dir not in sys.path:
    sys.path.insert(0, backtest_dir)
    
from Backtest import *
from BayesianOptimization import *
from ADX_RSIStrategy import ADX_RSIStrategy
from Backtest.open_json_gz_files import open_json_gz_files
from Backtest.generate_bars import generate_bars

In [2]:
def run_backtest(config, trading_data, ohlc_data, window_ADX = 10, window_RSI = 10, s=70, b=30):
    window_ADX = int(window_ADX)
    window_RSI = int(window_RSI)
    s = int(s)
    b = int(b)
    config['title'] = "ADX_RSIStrategy" + "_" + str(window_ADX) + "_" + str(window_RSI) + "_" + str(s) + "_" + str(b)
    print("---------------------------------")
    print(config['title'])
    print("---------------------------------")
    
    events_queue = queue.Queue()

    data_handler = OHLCDataHandler(
        config, events_queue,
        trading_data = trading_data, ohlc_data = ohlc_data
    )
    strategy = ADX_RSIStrategy(config, events_queue, data_handler,
                           window_ADX = window_ADX,
                            window_RSI=window_RSI, s=s, b=b)

    backtest = Backtest(config, events_queue, strategy,
                        data_handler= data_handler)

    results = backtest.start_trading()
    
    return (results['cum_returns'][-1] - 1)

## Setting

In [3]:
config = {
    "csv_dir": "C:/backtest/Binance",
    "out_dir": "C:/backtest/results/ADX_RSIStrategy",
    "title": "ADX_RSIStrategy",
    "is_plot": False,
    "save_plot": False,
    "save_tradelog": False,
    "start_date": pd.Timestamp("2017-07-01T00:0:00", freq = "60" + "T"),    # str(freq) + "T"
    "end_date": pd.Timestamp("2018-04-01T00:00:00", freq = "60" + "T"),
    "equity": 1.0,
    "freq": 60,      # min
    "commission_ratio": 0.001,
    "suggested_quantity": None,     # None or a value
    "max_quantity": None,           # None or a value, Maximum purchase quantity
    "min_quantity": None,           # None or a value, Minimum purchase quantity
    "min_handheld_cash": None,      # None or a value, Minimum handheld funds
    "exchange": "Binance",
    "tickers": ['BTCUSDT']
}

In [4]:
# trading_data = {}
# for ticker in config['tickers']:
#     # trading_data[ticker] = open_gz_files(config['csv_dir'], ticker)
#     trading_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker + '.h5', key=ticker)

ohlc_data = {}
for ticker in config['tickers']:
    # ohlc_data[ticker] = generate_bars(trading_data, ticker, config['freq'])
    ohlc_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker +'_OHLC_60min.h5', key=ticker)

trading_data = None

In [5]:
gp_params = {"alpha": 1e-5}

## Acquisition Function "Upper Confidence Bound"

* Prefer exploitation (kappa=1.0)

* Prefer exploration (kappa=10)


## Acquisition Function "Expected Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)

In [6]:
BO = BayesianOptimization(
    run_backtest,
    {'window_ADX': (1, 240),
    'window_RSI': (1, 120),
       's': (55, 85),
      'b': (15, 45)},
    is_int = [1, 1, 1, 1], 
    invariant = {
        'config': config,
        'trading_data': trading_data,
        'ohlc_data': ohlc_data
    },
    random_state = 1
)
# BO.explore({
#     'short_window': np.arange(1, 120, 20),
#     'delta_window': np.arange(1, 120, 20)
#     },
#     eager=True)
BO.maximize(init_points=50, n_iter=100, acq="ei", xi=0.01, **gp_params)

Initialization
-----------------------------------------------------------------------------------
 Step |   Time |      Value |         b |         s |   window_ADX |   window_RSI | 
---------------------------------
ADX_RSIStrategy_38_48_70_43
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.1234352030
Sortino Ratio: -0.2883396216
Max Drawdown: 0.0546148786
Max Drawdown Duration: 371500
Total Returns: -0.0292169002
Annualized Returns: -0.0687641176
Compound Annual Growth Rate: -0.0674091073
---------------------------------
Trades: 3
Trade Winning: 33.33%
Average Trade: -0.76%
Average Win: 1.78%
Average Loss: -2.04%
Best Trade: 1.78%
Worst Trade: -3.26%
Worst Trade Date: 0 days 04:00:00
Avg Days in Trade: 0 days 0

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
   10 | 00m07s |    0.00000 | 27.000000 | 55.000000 | 145.000000 | 58.000000 | 
---------------------------------
ADX_RSIStrategy_130_4_79_16
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.5584928480
Sortino Ratio

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.5296220415
Sortino Ratio: nan
Max Drawdown: 0.0198598655
Max Drawdown Duration: 371500
Total Returns: -0.0101267138
Annualized Returns: -0.0238339637
Compound Annual Growth Rate: -0.0236708099
---------------------------------
Trades: 1
Trade Winning: 0.00%
Average Trade: -0.81%
Average Win: nan%
Average Loss: -0.81%
Best Trade: -0.81%
Worst Trade: -0.81%
Worst Trade Date: 0 days 08:00:00
Avg Days in Trade: 0 days 08:00:00
---------------------------------
   20 | 00m07s |   -0.01013 | 43.000000 | 84.000000 | 147.000000 | 110.000000 | 
---------------------------------
ADX_RSIStrategy_213_116_68_22
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
-----------------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
   30 | 00m08s |    0.00000 | 17.000000 | 66.000000 | 216.000000 | 88.000000 | 
---------------------------------
ADX_RSIStrategy_234_8_62_22
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 3.3726377635
Sortino Ratio:

---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
   40 | 00m08s |    0.00000 | 37.000000 | 62.000000 | 62.000000 | 101.000000 | 
---------------------------------
ADX_RSIStrategy_23_28_68_31
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backte

...

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  104 | 00m57s |    0.00000 | 15.000000 | 55.000000 | 40.000000 | 86.000000 | 
---------------------------------
ADX_RSIStrategy_1_76_55_15
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Dr

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.7195471759
Sortino Ratio: -1.3288764181
Max Drawdown: 0.3955168097
Max Drawdown Duration: 293900
Total Returns: -0.3072600609
Annualized Returns: -0.7231590901
Compound Annual Growth Rate: -0.5785264399
---------------------------------
Trades: 368
Trade Winning: 70.92%
Average Trade: 0.11%
Average Win: 0.72%
Average Loss: -1.38%
Best Trade: 4.55%
Worst Trade: -15.84%
Worst Trade Date: 0 days 05:00:00
Avg Days in Trade: 0 days 01:39:37
---------------------------------
  113 | 00m49s |   -0.30726 | 30.000000 | 67.000000 | 236.000000 | 1.000000 | 
---------------------------------
ADX_RSIStrategy_221_21_71_20
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
-------

---------------------------------
ADX_RSIStrategy_40_1_85_15
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -0.6484359014
Sortino Ratio: -0.6438400076
Max Drawdown: 0.3532903302
Max Drawdown Duration: 293900
Total Returns: -0.1924551631
Annualized Returns: -0.4529573425
Compound Annual Growth Rate: -0.3953418800
---------------------------------
Trades: 467
Trade Winning: 71.52%
Average Trade: 0.17%
Average Win: 0.85%
Average Loss: -1.55%
Best Trade: 4.55%
Worst Trade: -15.84%
Worst Trade Date: 0 days 05:00:00
Avg Days in Trade: 0 days 01:32:53
---------------------------------
  123 | 00m53s |   -0.19246 | 15.000000 | 85.000000 | 40.000000 | 1.000000 | 
---------------------------------
ADX_RSIStrategy_240_120_85_15

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -2.6196249085
Sortino Ratio: -1.0742306531
Max Drawdown: 0.2505884578
Max Drawdown Duration: 342500
Total Returns: -0.2075342536
Annualized Returns: -0.4884470880
Compound Annual Growth Rate: -0.4215801405
---------------------------------
Trades: 10
Trade Winning: 40.00%
Average Trade: -2.05%
Average Win: 0.40%
Average Loss: -3.68%
Best Trade: 1.09%
Worst Trade: -10.61%
Worst Trade Date: 1 days 08:00:00
Avg Days in Trade: 0 days 14:48:00
---------------------------------
  132 | 01m06s |   -0.20753 | 45.000000 | 55.000000 | 240.000000 | 80.000000 | 
---------------------------------
ADX_RSIStrategy_22_61_85_15
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
-------

---------------------------------
ADX_RSIStrategy_39_120_85_45
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.1696685819
Sortino Ratio: 1.1114565794
Max Drawdown: 0.2072699619
Max Drawdown Duration: 244100
Total Returns: 0.1592433397
Annualized Returns: 0.3747908802
Compound Annual Growth Rate: 0.4159230452
---------------------------------
Trades: 21
Trade Winning: 42.86%
Average Trade: 1.06%
Average Win: 5.59%
Average Loss: -2.34%
Best Trade: 18.73%
Worst Trade: -7.35%
Worst Trade Date: 0 days 01:00:00
Avg Days in Trade: 0 days 19:05:43
---------------------------------
  142 | 01m04s |    0.15924 | 45.000000 | 85.000000 | 39.000000 | 120.000000 | 
---------------------------------
ADX_RSIStrategy_118_95_85_15
--

In [7]:
print(BO.res['max'])

{'max_val': 0.3206572783510666, 'max_params': {'window_ADX': 234.0, 'window_RSI': 8.0, 's': 62.0, 'b': 22.0}}


In [8]:
BO.maximize(init_points=0, n_iter=110, acq="ei", xi=0.0001, **gp_params)

Bayesian Optimization
-----------------------------------------------------------------------------------
 Step |   Time |      Value |         b |         s |   window_ADX |   window_RSI | 
---------------------------------
ADX_RSIStrategy_139_36_85_45
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -2.3712455239
Sortino Ratio: -1.4183721425
Max Drawdown: 0.4590122658
Max Drawdown Duration: 292400
Total Returns: -0.3683159863
Annualized Returns: -0.8668586890
Compound Annual Growth Rate: -0.6607953094
---------------------------------
Trades: 21
Trade Winning: 33.33%
Average Trade: -1.87%
Average Win: 2.29%
Average Loss: -3.95%
Best Trade: 7.72%
Worst Trade: -11.21%
Worst Trade Date: 0 days 21:00:00
Avg Days in Trade

C:\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-3.58863617e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 55, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


---------------------------------
ADX_RSIStrategy_78_1_55_45
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.4113595593
Sortino Ratio: -1.3582459954
Max Drawdown: 0.4364439754
Max Drawdown Duration: 293900
Total Returns: -0.3223193825
Annualized Returns: -0.7586023080
Compound Annual Growth Rate: -0.5997741871
---------------------------------
Trades: 435
Trade Winning: 70.11%
Average Trade: 0.12%
Average Win: 0.82%
Average Loss: -1.50%
Best Trade: 4.55%
Worst Trade: -15.84%
Worst Trade Date: 0 days 05:00:00
Avg Days in Trade: 0 days 01:35:02
---------------------------------
  159 | 00m57s |   -0.32232 | 45.000000 | 55.000000 | 78.000000 | 1.000000 | 
---------------------------------
ADX_RSIStrategy_240_120_55_15

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  168 | 01m01s |    0.00000 | 15.000000 | 55.000000 | 40.000000 | 31.000000 | 
---------------------------------
ADX_RSIStrategy_217_32_85_28
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.3587905146
Sortino Ratio

C:\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-3.11953421e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 54, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


...

C:\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-8.27105616e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 60, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


---------------------------------
ADX_RSIStrategy_172_76_55_24
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  240 | 01m46s |    0.00000 | 24.000000 | 55.000000 | 172.000000 | 76.000000 | 
---------------------------------
ADX_RSIStrategy_171_104_55_32
---------------------------------
Data Time Interval for BTCUSD

C:\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00012265]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 60, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-5.90440284e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 58, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


---------------------------------
ADX_RSIStrategy_196_104_85_25
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  243 | 01m42s |    0.00000 | 25.000000 | 85.000000 | 196.000000 | 104.000000 | 
---------------------------------
ADX_RSIStrategy_1_1_85_45
---------------------------------
Data Time Interval for BTCUSDT:

---------------------------------
ADX_RSIStrategy_36_57_55_37
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  252 | 01m56s |    0.00000 | 37.000000 | 55.000000 | 36.000000 | 57.000000 | 
---------------------------------
ADX_RSIStrategy_38_19_55_28
---------------------------------
Data Time Interval for BTCUSDT:
	

In [9]:
print(BO.res['max'])

{'max_val': 0.45809089108317647, 'max_params': {'window_ADX': 36.0, 'window_RSI': 96.0, 's': 55.0, 'b': 45.0}}


In [10]:
Target = pd.DataFrame({'Parameters': BO.X.tolist(), 'Target': BO.Y})
# Target.to_csv(config['out_dir'] + "/target_ei.csv")
Target.sort_values(by = "Target")

,Parameters,Target
75,"[1.0, 38.0, 85.0, 45.0]",-0.784957
177,"[1.0, 71.0, 55.0, 45.0]",-0.659330
67,"[1.0, 101.0, 85.0, 45.0]",-0.557936
162,"[1.0, 120.0, 55.0, 45.0]",-0.527672
153,"[168.0, 1.0, 55.0, 45.0]",-0.490606
219,"[5.0, 63.0, 63.0, 44.0]",-0.461386
145,"[192.0, 35.0, 85.0, 45.0]",-0.424759
237,"[124.0, 1.0, 55.0, 15.0]",-0.402268
60,"[178.0, 1.0, 85.0, 15.0]",-0.396888
204,"[218.0, 37.0, 74.0, 38.0]",-0.384893


 
 ## Acquisition Function "Probability of Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)